In [113]:
import os, math
import dask.dataframe as dd
import pandas as pd
import numpy as np
from scipy.sparse import lil_matrix, hstack

In [37]:
masterx2i, masteri2x = {}, {}
counter = 0

In [38]:
def createDict(file_name):
    global counter
    print(file_name)
    wc = pd.read_csv("wc2/wc_"+file_name+'.csv')
    wc = wc.drop("Unnamed: 0", axis=1)
    wc.columns = ["word", "count"]
    # print(wc)
    for index, row in wc.iterrows():
        
        if index % 100 == 0:
            print(f'finished indexing {index/len(wc):.2%} of vocab')
        
        if row["word"] not in masterx2i.keys():
            masterx2i[row["word"]] = counter
            masteri2x[counter] = row["word"]
            counter = counter+1
            # print(counter)

In [39]:
createDict("2018-01-22")

2018-01-22
finished indexing 0.00% of vocab
finished indexing 0.25% of vocab
finished indexing 0.50% of vocab
finished indexing 0.75% of vocab
finished indexing 1.00% of vocab
finished indexing 1.25% of vocab
finished indexing 1.50% of vocab
finished indexing 1.75% of vocab
finished indexing 2.00% of vocab
finished indexing 2.25% of vocab
finished indexing 2.50% of vocab
finished indexing 2.76% of vocab
finished indexing 3.01% of vocab
finished indexing 3.26% of vocab
finished indexing 3.51% of vocab
finished indexing 3.76% of vocab
finished indexing 4.01% of vocab
finished indexing 4.26% of vocab
finished indexing 4.51% of vocab
finished indexing 4.76% of vocab
finished indexing 5.01% of vocab
finished indexing 5.26% of vocab
finished indexing 5.51% of vocab
finished indexing 5.76% of vocab
finished indexing 6.01% of vocab
finished indexing 6.26% of vocab
finished indexing 6.51% of vocab
finished indexing 6.76% of vocab
finished indexing 7.01% of vocab
finished indexing 7.26% of vocab

finished indexing 62.37% of vocab
finished indexing 62.62% of vocab
finished indexing 62.87% of vocab
finished indexing 63.12% of vocab
finished indexing 63.37% of vocab
finished indexing 63.62% of vocab
finished indexing 63.87% of vocab
finished indexing 64.12% of vocab
finished indexing 64.37% of vocab
finished indexing 64.62% of vocab
finished indexing 64.87% of vocab
finished indexing 65.12% of vocab
finished indexing 65.37% of vocab
finished indexing 65.62% of vocab
finished indexing 65.88% of vocab
finished indexing 66.13% of vocab
finished indexing 66.38% of vocab
finished indexing 66.63% of vocab
finished indexing 66.88% of vocab
finished indexing 67.13% of vocab
finished indexing 67.38% of vocab
finished indexing 67.63% of vocab
finished indexing 67.88% of vocab
finished indexing 68.13% of vocab
finished indexing 68.38% of vocab
finished indexing 68.63% of vocab
finished indexing 68.88% of vocab
finished indexing 69.13% of vocab
finished indexing 69.38% of vocab
finished index

In [46]:
wc = pd.read_csv("./wc2/wc_2018-01-22.csv")
wnd = pd.read_csv("./wnd2/wnd_2018-01-22.csv")

In [124]:
def delete_row_lil(mat, i):
    if not isinstance(mat, lil_matrix):
        raise ValueError("works only for LIL format -- use .tolil() first")
    mat.rows = np.delete(mat.rows, i)
    mat.data = np.delete(mat.data, i)
    mat._shape = (mat._shape[0] - 1, mat._shape[1])

In [146]:
def createMat(wc, wnd):
    
    curr_mat = lil_matrix(np.zeros((1,len(masterx2i)))) 
    
    for index, word1 in masteri2x.items():
        
        temp_row = np.zeros((1,len(masterx2i))) # the row in ppmi matrix
        
        for i in range(len(masteri2x)):
            
            word2 = masteri2x[i]
            
            wc1row = wc.loc[wc['0']==word1].values.tolist()
            wc2row = wc.loc[wc['0']==word2].values.tolist()
            wc1 = wc1row[0][2]   # word count word1
            wc2 = wc2row[0][2]   # word count word2
            
            wndw1q = wnd.loc[wnd['0']==word1]
            wndw2q = wndw1q.loc[wndw1q['1']==word2].values.tolist() # gets row for window count     
            try:
                wndc = wndw2q[0][3]
            except IndexError:    # if the words do not occur together then ppmi val should be 0
                temp_row[0][i] = 0
                break
            
            top = wndc * len(masteri2x)
            bot = wc1 * wc2
            val = math.log(float(top/bot),2) # calculate ppmi val
            
            if val < 0:
                val = 0
                
            temp_row[0][i] = val
        
        sm_temp = lil_matrix(temp_row)
        print(sm_temp.ndim)
        print(curr_mat.ndim)
        curr_mat = hstack(curr_mat, sm_temp) # stacks rows in top of each other
    
    delete_row_lil(curr_mat.tolil(), [-1])
    return curr_mat

In [147]:
ppmi = createMat(wc, wnd)

2
2


ValueError: blocks must be 2-D